In [1]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals,
    Sector
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
#from sharadar.pipeline.universes import TRADABLE_STOCKS_US, NamedUniverse
from sharadar.pipeline.factors import Exchange, Sector, IsDomesticCommonStock, MarketCap, Fundamentals, EV, EvEbit


def stocks_us():
    return (
            (USEquityPricing.close.latest > 3) &
            Exchange().element_of(['NYSE', 'NASDAQ', 'NYSEMKT']) &
            (Sector().notnull()) &
            (~Sector().element_of(['Financial Services', 'Real Estate', 'Energy', 'Utilities'])) &
            (IsDomesticCommonStock().eq(1)) &
            (Fundamentals(field='revenue_arq') > 0) &
            (Fundamentals(field='assets_arq') > 0) &
            (Fundamentals(field='equity_arq') > 0) &
            (EV() > 0)
    )

In [9]:
engine = make_pipeline_engine()

pipe_date = pd.to_datetime('2024-07-01', utc=False)
#pipe_date_end = pd.to_datetime('2024-02-05', utc=False)

pipe = Pipeline(columns={
    'mkt_cap': MarketCap(),
    'ev': EV(),
    'netmargin': Fundamentals(field='netmargin'),
    'revenue': Fundamentals(field='revenue'),
    'EvEbit' : EvEbit(),
    'sector': Sector(),
    }, screen = stocks_us()
#,screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
#,screen = NamedUniverse(TRADABLE_STOCKS_US)
#screen = TradableStocksUS()
    )

df = engine.run_pipeline(pipe, pipe_date)

[2024-07-12 16:30:42] INFO: Load root mask file: root-2024-07-01_2024-07-01_XNYS_US_0.pkl
Pipeline from 2024-07-01 to 2024-07-01
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_mkt_cap.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_ev.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_netmargin.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_revenue.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_EvEbit.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_sector.npy to cache
[2024-07-12 16:31:00] INFO: save term-2024-07-01_2024-07-01_screen_c6d76a04b00d449680be293298a28298_screen_c6d76a04b00d449680be293298a28298.npy to cac

In [15]:
df.head()

mkt_cap           ev  netmargin  \
2024-07-01 Equity(108199 [ATLX])  172100000.0  165000000.0    -69.432   
           Equity(108216 [AUID])   94500000.0   87500000.0    -19.428   
           Equity(108271 [MAMA])  241500000.0  241000000.0      0.019   
           Equity(108437 [BRFH])   53300000.0   52100000.0     -0.159   
           Equity(108479 [BTTR])    3300000.0    4800000.0     -0.358   

                                     revenue  EvEbit              sector  
2024-07-01 Equity(108199 [ATLX])    186707.0    -3.3     Basic Materials  
           Equity(108216 [AUID])    157378.0    -4.3          Technology  
           Equity(108271 [MAMA])  29838000.0    30.6  Consumer Defensive  
           Equity(108437 [BRFH])   2829000.0   -22.0  Consumer Defensive  
           Equity(108479 [BTTR])   7903000.0    -0.2  Consumer Defensive

In [16]:
df_group_by_sector = df.groupby('sector', observed=True)
sector_size = df_group_by_sector.size()
sector_size

sector
Basic Materials           151
Communication Services     97
Consumer Cyclical         312
Consumer Defensive        138
Healthcare                396
Industrials               400
Technology                428
dtype: int64

In [17]:
# There are Sectors with size less than 30?
len(sector_size[sector_size < 30]) > 0

False

In [18]:
sector_rank = df_group_by_sector.rank(ascending=False, pct=True)
sector_rank.sort_values(by='netmargin', ascending=False).head()

mkt_cap        ev  netmargin  revenue  \
2024-07-01 Equity(124773 [ATOM])  0.900701  0.913551        1.0  1.00000   
           Equity(165489 [RKDA])  0.992754  1.000000        1.0  1.00000   
           Equity(631233 [VRM])   0.993590  0.717949        1.0  0.99359   
           Equity(631977 [JOBY])  0.427500  0.560000        1.0  1.00000   
           Equity(636114 [DJT])   0.350515  0.402062        1.0  1.00000   

                                    EvEbit  
2024-07-01 Equity(124773 [ATOM])  0.655738  
           Equity(165489 [RKDA])  0.829710  
           Equity(631233 [VRM])   0.852564  
           Equity(631977 [JOBY])  0.887500  
           Equity(636114 [DJT])   0.783505

In [19]:
score = sector_rank.mean(axis=1)
score.head()

2024-07-01  Equity(108199 [ATLX])    0.940397
            Equity(108216 [AUID])    0.892828
            Equity(108271 [MAMA])    0.689855
            Equity(108437 [BRFH])    0.952174
            Equity(108479 [BTTR])    0.954348
dtype: float64

In [20]:
score

2024-07-01  Equity(108199 [ATLX])    0.940397
            Equity(108216 [AUID])    0.892828
            Equity(108271 [MAMA])    0.689855
            Equity(108437 [BRFH])    0.952174
            Equity(108479 [BTTR])    0.954348
                                       ...   
            Equity(641911 [FLYE])    0.630769
            Equity(641912 [WAY])     0.298232
            Equity(641966 [GRAL])    0.703788
            Equity(641986 [WBTN])    0.538461
            Equity(641989 [LSH])     0.778000
Length: 1922, dtype: float64